In [1]:
import ray
from ray import rllib, tune
import pandas as pd
import numpy as np
from datetime import datetime
import trading_gym
from trading_gym.registry.gaia.v9.env import GAIAPredictorsContinuousV9
from trading_gym.ray.walkforward import WalkForwardRunner, WalkForwardResults
%matplotlib inline
print(trading_gym.__package__, trading_gym.__version__)
print(ray.__package__, ray.__version__)

/home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/Nicholas/trading

trading-gym 0.8.1
ray 0.7.3


In [2]:
from trading_gym.registry.gaia.v11.env import GAIAPredictorsContinuousV11
# from trading_gym.registry.gaia.v12.env import GAIAPredictorsContinuousV12

In [3]:
ray.init()

2019-08-24 23:30:52,776	INFO node.py:498 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-24_23-30-52_773639_101236/logs.
2019-08-24 23:30:52,964	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:54747 to respond...
2019-08-24 23:30:53,110	INFO services.py:409 -- Waiting for redis server at 127.0.0.1:21000 to respond...
2019-08-24 23:30:53,120	INFO services.py:809 -- Starting Redis shard with 10.0 GB max memory.
2019-08-24 23:30:53,195	INFO node.py:512 -- Process STDOUT and STDERR is being redirected to /tmp/ray/session_2019-08-24_23-30-52_773639_101236/logs.
2019-08-24 23:30:53,203	WARNING services.py:1301 -- Warning: Capping object memory store to 20.0GB. To increase this further, specify `object_store_memory` when calling ray.init() or ray start.
2019-08-24 23:30:53,204	WARNING services.py:1330 -- WARNING: The default object store size of 20.0 GB will use more than 50% of the available memory on this node (27.74 GB). Consider setting the objec

{'node_ip_address': '10.0.5.4',
 'redis_address': '10.0.5.4:54747',
 'object_store_address': '/tmp/ray/session_2019-08-24_23-30-52_773639_101236/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2019-08-24_23-30-52_773639_101236/sockets/raylet',
 'webui_url': None,
 'session_dir': '/tmp/ray/session_2019-08-24_23-30-52_773639_101236'}

## Create the walk-forward partitions
It's responsibility of the user to create the training/test (and maybe validation) partitions to turn the walk forward training. Note that 2-fold split is a particular case of walk-forward training, so you are still able to run a simple 2-fold split.


In [4]:
partitions = list()
for year in range(2007, 2018):
    partition = {
        'training-set': [datetime.min, datetime(year, 12, 31)],
        'test-set': [datetime(year + 1, 1, 1), datetime(year + 1, 12, 31)],
    }
    partitions.append(partition)
partitions


[{'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2007, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2008, 1, 1, 0, 0),
   datetime.datetime(2008, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2008, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2009, 1, 1, 0, 0),
   datetime.datetime(2009, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2009, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2010, 1, 1, 0, 0),
   datetime.datetime(2010, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2010, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2011, 1, 1, 0, 0),
   datetime.datetime(2011, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 1, 1, 0, 0),
   datetime.datetime(2011, 12, 31, 0, 0)],
  'test-set': [datetime.datetime(2012, 1, 1, 0, 0),
   datetime.datetime(2012, 12, 31, 0, 0)]},
 {'training-set': [datetime.datetime(1, 

In [5]:
## Create the config dict
config = ray.rllib.agents.ppo.DEFAULT_CONFIG.copy()
config['env'] = GAIAPredictorsContinuousV11
config['env_config'] = {
    'cost_of_commissions': tune.grid_search([0.00005]),
    'cost_of_spread': 0.0001,
}
config['gamma'] = tune.grid_search([0.95]) # 2 weeks

In [6]:
config['vf_clip_param'] = 0
config['vf_loss_coeff'] = 0
config['lambda'] = 0
config['use_gae'] = True

# need to have vf share layers if lstm is used
# config['vf_share_layers'] = True
# config['model']['use_lstm'] = True
config['batch_mode'] = 'complete_episodes'
config['train_batch_size'] = 4000 # tune.grid_search([4000])
config['sgd_minibatch_size'] = 128
config['num_sgd_iter'] = tune.grid_search([8])
config['entropy_coeff'] =  1e-5 # tune.grid_search([1e-5])
config['kl_coeff'] = 0.2 #tune.grid_search([0.2])
config['kl_target'] = tune.grid_search([0.01])
config['clip_param'] = tune.grid_search([0.8])

config['lr'] = tune.grid_search([1e-5])

# config['model']['custom_model'] = CNN.__name__


In [7]:
# env = GAIAPredictorsContinuousV9()
# length = env.episode_length()

In [8]:
env = GAIAPredictorsContinuousV11()
env.action_space.sample()

/home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/trading_gym/transmitter.py:399: FutureWarning:

The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'



array([-0.06164829,  0.04413464])

In [9]:
## Run your walk-forward experiment
walk_forward = WalkForwardRunner(
    env_partitions=partitions,
    trainable=ray.rllib.agents.ppo.PPOTrainer,
    config=config,
    stop={'timesteps_total': 500000},
    checkpoint_freq=1,
)

Note that WalkForwardRunner has constructed the implied ray Experiment(s) from your walk forward settings.

walk_forward.experiments

Note that trials are associated with a `RestoreID`. This `ID` is all you need to restore an agent. Here we are using a grid search of two values for `cost_of_commissions` on two partitions, so we have a total of 4 experiments.

In [10]:
trials = walk_forward.run(verbose=0)
trials

2019-08-24 23:30:57,999	WARNING experiment.py:223 -- All experiments will be using the same SearchAlgorithm.
2019-08-24 23:30:58,015	INFO trial_runner.py:176 -- Starting a new experiment.
2019-08-24 23:30:58,059	WARNING signature.py:108 -- The function with_updates has a **kwargs argument, which is currently not supported.
2019-08-24 23:30:58,085	ERROR log_sync.py:34 -- Log sync requires cluster to be setup with `ray up`.
2019-08-24 23:30:58,193	WARNING util.py:145 -- The `start_trial` operation took 0.144789457321167 seconds to complete, which may be a performance bottleneck.


(pid=101362) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=101362)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=101362) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=101362)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=101362) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=101

(pid=101368) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/trading_gym/transmitter.py:399: FutureWarning:
(pid=101368) 
(pid=101368) The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
(pid=101368) 
(pid=101365) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/trading_gym/transmitter.py:399: FutureWarning:
(pid=101365) 
(pid=101365) The 'contains' method is deprecated and will be removed in a future version. Use 'key in index' instead of 'index.contains(key)'
(pid=101365) 
(pid=101368) 2019-08-24 23:31:16,530	INFO rollout_worker.py:319 -- Creating policy evaluation worker 1 on CPU (please ignore any CUDA init errors)
(pid=101368) 2019-08-24 23:31:16.558755: I tensorflow/core/platform/cpu_feature_guard.cc:141] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
(pid=101365) 2019-08-24 23:31:16,684	INFO rollout_worker.py:319 

(pid=101367) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=101367)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=101367) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=101367)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=101367) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=101

(pid=101364) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:523: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=101364)   _np_qint8 = np.dtype([("qint8", np.int8, 1)])
(pid=101364) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:524: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=101364)   _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
(pid=101364) /home/Nicholas/trading-gym_0.8.1/venv0.8.1/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:525: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
(pid=101

...

2019-08-25 07:00:47,038	WARNING util.py:145 -- The `process_trial` operation took 0.11930155754089355 seconds to complete, which may be a performance bottleneck.


(pid=21755) 2019-08-25 07:01:03,287	WARNING ppo.py:121 -- The magnitude of your environment rewards are more than infx the scale of `vf_clip_param`. This means that it will take more than inf iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
(pid=21755) 2019-08-25 07:01:22,820	WARNING ppo.py:121 -- The magnitude of your environment rewards are more than infx the scale of `vf_clip_param`. This means that it will take more than inf iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
(pid=21755) 2019-08-25 07:01:38,873	WARNING ppo.py:121 -- The magnitude of your environment rewards are more than infx the scale of `vf_clip_param`. This means that it will take more than inf iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.
(pid=21755) 2019-08-25 07:01:54,936	WARNING ppo.py:121 -- The magnitude of your environment re

2019-08-25 07:05:45,182	WARNING util.py:145 -- The `process_trial` operation took 0.10146784782409668 seconds to complete, which may be a performance bottleneck.


(pid=21755) 2019-08-25 07:05:57,779	WARNING ppo.py:121 -- The magnitude of your environment rewards are more than infx the scale of `vf_clip_param`. This means that it will take more than inf iterations for your value function to converge. If this is not intended, consider increasing `vf_clip_param`.


[PPO_GAIAPredictorsContinuousV11_restoreID=5710329639900672484_runID=TFAyTCuz_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV11_restoreID=5710329639900672484_runID=TFAyTCuz_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV11_restoreID=5710329639900672484_runID=TFAyTCuz_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV11_restoreID=5710329639900672484_runID=TFAyTCuz_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV11_restoreID=5710329639900672484_runID=TFAyTCuz_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95,kl_target=0.01,lr=1e-05,num_sgd_iter=8,
 PPO_GAIAPredictorsContinuousV11_restoreID=5710329639900672484_runID=TFAyTCuz_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.95,kl_targe

In [ ]:
# stop

# Temp

In [ ]:
agent = ray.rllib.agents.ppo.PPOTrainer(config, GAIAPredictorsContinuousV11)

In [ ]:
env = GAIAPredictorsContinuousV9()
state = env.reset()
state

In [ ]:
agent.compute_action(state)

# Restore

In [16]:
results = WalkForwardResults('/home/Nicholas/trading-gym_0.8.1/trading-gym/notebooks/registry/gaia/v9/logs')
results

WalkForwardResults(['GAIAPredictorsContinuousV11', 'GAIAPredictorsContinuousV12'])

In [17]:
env_results = results['GAIAPredictorsContinuousV11']
env_results

EnvResults(GAIAPredictorsContinuousV11)

In [18]:
# Step 1.
env = env_results.make_env(
    env_config={
        'cost_of_commissions': 0.00005,
        'cost_of_spread': 0.0001,
        'folds': {
            'training-set': [datetime.min, datetime(2008, 3, 18)],
            'test-set': [datetime(2008, 3, 19), datetime.max],
        }
    },
)

In [19]:
env_results.restore_ids

{-6695192150303784323: [AgentResults(GAIAPredictorsContinuousV11_1-01-01_to_2011-12-31/PPO_GAIAPredictorsContinuousV11_restoreID=-6695192150303784323_runID=bNl30CUT_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.9_2019-08-11_01-48-221_87dum0),
  AgentResults(GAIAPredictorsContinuousV11_1-01-01_to_2017-12-31/PPO_GAIAPredictorsContinuousV11_restoreID=-6695192150303784323_runID=bNl30CUT_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.9_2019-08-11_06-39-165dnrmr38),
  AgentResults(GAIAPredictorsContinuousV11_1-01-01_to_2008-12-31/PPO_GAIAPredictorsContinuousV11_restoreID=-6695192150303784323_runID=bNl30CUT_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.9_2019-08-10_23-22-37dumw8eu0),
  AgentResults(GAIAPredictorsContinuousV11_1-01-01_to_2016-12-31/PPO_GAIAPredictorsContinuousV11_restoreID=-6695192150303784323_runID=bNl30CUT_0_clip_param=0.8,cost_of_commissions=5e-05,gamma=0.9_2019-08-11_05-51-10jxzlovyl),
  AgentResults(GAIAPredictorsContinuousV11_1-01-01_to_2012-12-31/PPO_GAIAP

In [15]:
2557338427780778455
print(list(env_results.restore_ids)[-1])

2557338427780778455


In [25]:
restore_id = list(env_results.restore_ids)[-1]
restore_id = 5710329639900672484

In [26]:
policy = env_results.make_policy(
    env=env,
    restore_id=restore_id,
    checkpoint_nr=None,  # use None (or don't specify) to use last checkpoint available
)
policy

In [27]:
history = policy.history()
# history

In [28]:
# Step 3.
episode = env.sample_episode(fold='test-set', policy=policy, verbose=False)

2019-08-25 10:55:56,034	WARNING ppo.py:143 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
2019-08-25 10:55:57,228	INFO rollout_worker.py:319 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
2019-08-25 10:55:59,391	INFO rollout_worker.py:742 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7f29a292b630>}
2019-08-25 10:55:59,392	INFO rollout_worker.py:743 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7f29a292b208>}
2019-08-25 10:55:59,393	INFO rollout_worker.py:356 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7f2f8ce09e10>}
2019-08-25 10:55:59,488	INFO multi_gpu_optimizer.py:93 -- LocalMultiGPUOptimizer devices ['/cpu:0']
2019-08-25 10:56:05,744	WARNING util.py:47 -- Install gputil for GPU system monitoring.
2019-08-25 10:56:06,627	WARNING ppo

2019-08-25 10:57:18,918	INFO rollout_worker.py:742 -- Built policy map: {'default_policy': <ray.rllib.policy.tf_policy_template.PPOTFPolicy object at 0x7f297bc533c8>}
2019-08-25 10:57:18,920	INFO rollout_worker.py:743 -- Built preprocessor map: {'default_policy': <ray.rllib.models.preprocessors.NoPreprocessor object at 0x7f297bc7cf60>}
2019-08-25 10:57:18,921	INFO rollout_worker.py:356 -- Built filter map: {'default_policy': <ray.rllib.utils.filter.NoFilter object at 0x7f297bc7cd30>}
2019-08-25 10:57:18,977	INFO multi_gpu_optimizer.py:93 -- LocalMultiGPUOptimizer devices ['/cpu:0']
2019-08-25 10:57:23,942	WARNING util.py:47 -- Install gputil for GPU system monitoring.
2019-08-25 10:57:25,171	WARNING ppo.py:143 -- FYI: By default, the value function will not share layers with the policy model ('vf_share_layers': False).
2019-08-25 10:57:26,333	INFO rollout_worker.py:319 -- Creating policy evaluation worker 0 on CPU (please ignore any CUDA init errors)
2019-08-25 10:57:28,340	INFO rollou

In [29]:
episode.renderer.cumulative_performance.to_plotly()
episode.renderer.target_weights.to_plotly()
episode.renderer.annual_returns.to_plotly()
episode.renderer.tearsheet()

Strategy  \
Context              From                              2008-03-19   
                     To                                2018-08-28   
                     Years                                10.4493   
                     Observations                            2725   
                     Risk-free asset        Index(USD 1M Deposit)   
                     Risk-free CAGR                    0.00681294   
Return               CAGR                               0.0789021   
                     CAGR over cash                     0.0720891   
                     Overall return                       1.21126   
Risk                 Volatility                         0.0905925   
                     Downside volatility                0.0638477   
                     Upside volatility                  0.0641182   
                     Max drawdown                        -0.12131   
                     Martin risk                        0.0467706   
                     VaR 5%                           -0.00885811   
                     VaR 2%                            -0.0128723   
                     Expected shortfall 5%             -0.0130935   
                     Expected shortfall 2%             -0.0166408   
Risk-adjusted return Sharpe ratio                        0.795752   
                     Sortino ratio                        1.12908   
                     Calmar ratio                        0.594254   
                     Martin ratio                         1.54134   
Outperformance       Benchmark name         Index(Aric-Benchmark)   
                     CAGR over benchmark               -0.0796835   
                     Information ratio                  -0.965598   
                     CAPM Alpha                       -0.00964166   
                     CAPM Beta                           0.574104   
Weights              Cash(USD)                         0.00401481   
                     ETF(Russell 1000)                   0.356497   
                     ETF(7-10Y T-Bills)                  0.639488   
                     Leverage mean                              1   
                     Turnover daily                      0.028454   
                     Turnover annual                      7.17041   

                                            Index(Aric-Benchmark)  \
Context              From                              2008-03-19   
                     To                                2018-08-28   
                     Years                                10.4493   
                     Observations                            2725   
                     Risk-free asset        Index(USD 1M Deposit)   
                     Risk-free CAGR                    0.00681294   
Return               CAGR                                0.158586   
                     CAGR over cash                      0.151773   
                     Overall return                       3.65592   
Risk                 Volatility                         0.0970738   
                     Downside volatility                 0.069906   
                     Upside volatility                  0.0705399   
                     Max drawdown                      -0.0865477   
                     Martin risk                        0.0189993   
                     VaR 5%                           -0.00895352   
                     VaR 2%                            -0.0134313   
                     Expected shortfall 5%             -0.0137858   
                     Expected shortfall 2%              -0.017882   
Risk-adjusted return Sharpe ratio                         1.56348   
                     Sortino ratio                         2.1711   
                     Calmar ratio                         1.75363   
                     Martin ratio                         7.98834   
Outperformance       Benchmark name         Index(Aric-Benchmark)   
                     CAGR over benchmark              

In [30]:
# cost2restore_id = {
#     0.00001: -3264367374635941251,    
# }

In [21]:
# nr2episode = dict()
# for cost_of_commissions, restore_id in cost2restore_id.items():
#     nr2episode[cost_of_commissions] = env_results.get_nr2episode(
#         restore_id=restore_id,
#         checkpoint_nrs=np.arange(1, 126, 1),
#         fold='test-set',
#         env_config={
#             'folds': {
#                 'training-set': [datetime.min, datetime(2008, 3, 18)],
#                 'test-set': [datetime(2008, 3, 19), datetime.max],
#             }
#         }
#     )

In [ ]:
cost_of_commissions = 0.00001
nr2episode[cost_of_commissions].plot_weights()

In [ ]:
nr2episode[cost_of_commissions].plot_levels()

In [ ]:
nr2episode[cost_of_commissions].plot_metrics_as_we_train()